In [1]:
### THIS IS THE SAME AS WHAT WAS DONE FOR ALL OF THE OTHER DATA
###... DC HAD TO BE DONE SEPARATELY BECAUSE IT IS NOT WITHIN A COUNTY
###... make sure you've read through the cleaning_health_data_final notebook if you dont understand a step

import pandas as pd
import regex as re

In [4]:
df = pd.read_csv("./datasets/500_Cities__Local_Data_for_Better_Health__2017_release.csv")

In [5]:
dc = df[df["StateAbbr"] == "DC"]

In [6]:
dc_census = dc[dc['GeographicLevel'] == "Census Tract"]

In [7]:
re.findall("-\d+.\d+", dc_census.GeoLocation[230262])[0] # this is how the regex will work for Long

'-77.0607838311'

In [8]:
dc_census['Latitude'] = [float(re.findall("(\d+.\d+),", dc_census.GeoLocation[i])[0]) for i in dc_census.index] # regex for lat

/home/ryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
dc_census['Longitude'] = [float(re.findall("-\d+.\d+", dc_census.GeoLocation[i])[0]) for i in dc_census.index] # regex for long

/home/ryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
dc_census.Measure.value_counts()

Visits to doctor for routine checkup within the past Year among adults aged >=18 Years                                                                                                            179
High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years                                                                                                          179
Binge drinking among adults aged >=18 Years                                                                                                                                                       179
Current lack of health insurance among adults aged 18–64 Years                                                                                                                                    179
Current asthma among adults aged >=18 Years                                                                                                                                                       179
Physical h

In [11]:
dc_census.Measure.value_counts().index[0]

'Visits to doctor for routine checkup within the past Year among adults aged >=18 Years'

In [12]:
measures_of_interest = df.Measure.value_counts().index

In [13]:
dc_df_list = [dc_census[dc_census["Measure"] == x] for x in measures_of_interest] # making a df for each measure

In [14]:
metric_dfs = []
for df in dc_df_list:
    df.reset_index(inplace=True, drop=True)
    temp = pd.DataFrame({df.MeasureId[0]:df.Data_Value, "UniqueID":df.UniqueID,
                                       "geometry":df.GeoLocation, "state":df.StateAbbr,
                            "pop":df.PopulationCount, "longitude": df.Longitude,
                           "latitude":df.Latitude})
    metric_dfs.append(temp)

In [15]:
metric_dfs[0].UniqueID[3][0:5]

'11500'

In [16]:
metric_dfs[0].UniqueID.value_counts().sum()

179

In [17]:
master = pd.merge(metric_dfs[0],metric_dfs[1], how='left',on="UniqueID")

In [18]:
for i in metric_dfs[2:]:
    master = master.merge(i, how = "left",on="UniqueID")

In [19]:
master.columns

Index(['DENTAL', 'UniqueID', 'geometry_x', 'state_x', 'pop_x', 'longitude_x',
       'latitude_x', 'SLEEP', 'geometry_y', 'state_y',
       ...
       'state_x', 'pop_x', 'longitude_x', 'latitude_x', 'COREW', 'geometry_y',
       'state_y', 'pop_y', 'longitude_y', 'latitude_y'],
      dtype='object', length=169)

In [20]:
master = master.T.drop_duplicates().T

In [24]:
master.columns

Index(['DENTAL', 'UniqueID', 'geometry_x', 'state_x', 'pop_x', 'longitude_x',
       'latitude_x', 'SLEEP', 'BPHIGH', 'COPD', 'CANCER', 'CHECKUP',
       'DIABETES', 'CSMOKING', 'CHOLSCREEN', 'PHLTH', 'STROKE', 'KIDNEY',
       'CHD', 'HIGHCHOL', 'CASTHMA', 'BINGE', 'LPA', 'BPMED', 'ARTHRITIS',
       'MHLTH', 'OBESITY', 'ACCESS2', 'PAPTEST', 'COLON_SCREEN', 'MAMMOUSE',
       'TEETHLOST', 'geometry', 'state', 'population', 'longitude', 'latitude',
       'COREM', 'COREW'],
      dtype='object')

In [21]:
master.columns

Index(['DENTAL', 'UniqueID', 'geometry_x', 'state_x', 'pop_x', 'longitude_x',
       'latitude_x', 'SLEEP', 'BPHIGH', 'COPD', 'CANCER', 'CHECKUP',
       'DIABETES', 'CSMOKING', 'CHOLSCREEN', 'PHLTH', 'STROKE', 'KIDNEY',
       'CHD', 'HIGHCHOL', 'CASTHMA', 'BINGE', 'LPA', 'BPMED', 'ARTHRITIS',
       'MHLTH', 'OBESITY', 'ACCESS2', 'PAPTEST', 'COLON_SCREEN', 'MAMMOUSE',
       'TEETHLOST', 'geometry_y', 'state_y', 'pop_y', 'longitude_y',
       'latitude_y', 'COREM', 'COREW'],
      dtype='object')

In [25]:
master.rename({"geometry_y":"geometry","state_y":"state","pop_y":"population","longitude_y":"longitude",
              "latitude_y":"latitude"}, axis = 1, inplace = True)
master.drop(['geometry_x', 'state_x', 'pop_x', 'longitude_x','latitude_x'], axis = 1, inplace = True)

In [26]:
master.dropna().shape, master.shape

((178, 34), (179, 34))

In [27]:
master.dropna(inplace=True)

In [28]:
import addfips
from progressbar import ProgressBar

In [29]:
master['fips'] = [x[0:5] for x in master.UniqueID]

In [32]:
master.head()

,DENTAL,UniqueID,SLEEP,BPHIGH,COPD,CANCER,CHECKUP,DIABETES,CSMOKING,CHOLSCREEN,...,MAMMOUSE,TEETHLOST,geometry,state,population,longitude,latitude,COREM,COREW,fips
0,86.6,1150000-11001000100,24.6,21.7,3.1,7.2,73.3,5,9.3,86.2,...,80,3.1,"(38.9060811983, -77.0607838311)",DC,4890,-77.0608,38.9061,41.3,39.1,11500
1,53.1,1150000-11001000201,34.2,9.1,4.7,1,61.1,1.7,27,39.4,...,63.2,21.6,"(38.9093385463, -77.0748289955)",DC,3916,-77.0748,38.9093,28.6,19.8,11500
2,81.5,1150000-11001000202,26.1,15.8,3.4,4.1,68.3,3.9,13.3,69.9,...,76.6,5.2,"(38.9061877176, -77.0696162292)",DC,5425,-77.0696,38.9062,38.1,34.9,11500
3,82.7,1150000-11001000300,26.3,14.9,2.8,4.1,67.7,3.4,12.4,75.7,...,78.3,4.3,"(38.9175532799, -77.0756590546)",DC,6233,-77.0757,38.9176,38.9,36,11500
4,83.2,1150000-11001000400,26.1,25.1,4,7.6,74.7,6.4,10.8,86.2,...,78.5,4.5,"(38.9238892355, -77.0658962319)",DC,1455,-77.0659,38.9239,38.3,36.6,11500


In [33]:
master.columns

Index(['DENTAL', 'UniqueID', 'SLEEP', 'BPHIGH', 'COPD', 'CANCER', 'CHECKUP',
       'DIABETES', 'CSMOKING', 'CHOLSCREEN', 'PHLTH', 'STROKE', 'KIDNEY',
       'CHD', 'HIGHCHOL', 'CASTHMA', 'BINGE', 'LPA', 'BPMED', 'ARTHRITIS',
       'MHLTH', 'OBESITY', 'ACCESS2', 'PAPTEST', 'COLON_SCREEN', 'MAMMOUSE',
       'TEETHLOST', 'geometry', 'state', 'population', 'longitude', 'latitude',
       'COREM', 'COREW', 'fips'],
      dtype='object')

In [51]:
master.to_csv("./dc_health_nonagg_final.csv")

In [38]:
agg_cols = ['DENTAL','SLEEP', 'BPHIGH', 'COPD', 'CANCER', 'CHECKUP',
       'DIABETES', 'CSMOKING', 'CHOLSCREEN', 'PHLTH', 'STROKE', 'KIDNEY',
       'CHD', 'HIGHCHOL', 'CASTHMA', 'BINGE', 'LPA', 'BPMED', 'ARTHRITIS',
       'MHLTH', 'OBESITY', 'ACCESS2', 'PAPTEST', 'COLON_SCREEN', 'MAMMOUSE',
       'TEETHLOST','COREM', 'COREW', "fips"]

In [42]:
clean_dc = master[agg_cols]

In [43]:
for col in clean_dc.dtypes.index:
    clean_dc[col] = clean_dc[col].astype(float)

/home/ryan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
dc_agg = clean_dc.groupby("fips").mean().reset_index(drop = False)

In [48]:
dc_agg

,fips,DENTAL,SLEEP,BPHIGH,COPD,CANCER,CHECKUP,DIABETES,CSMOKING,CHOLSCREEN,...,ARTHRITIS,MHLTH,OBESITY,ACCESS2,PAPTEST,COLON_SCREEN,MAMMOUSE,TEETHLOST,COREM,COREW
0,11500.0,67.076966,36.208427,30.154494,5.28764,5.15,74.712921,9.587079,19.527528,77.503933,...,19.858427,11.780899,26.36236,10.58427,83.691573,64.805056,77.757865,13.016292,29.223034,27.173034


In [49]:
dc_agg["County-equivalent"] = "DC"
dc_agg["unique_county"] = "WashingtonDC"
dc_agg["State"] = "DC"
dc_agg["Medianhouseholdincome"] = 65830
dc_agg["Medianfamilyincome"] = 79665
dc_agg["Per capitaincome"] = 45290
dc_agg["Population"] = 619371
dc_agg["Number ofhouseholds"] = 263649

In [50]:
dc_agg.to_csv("dc_final_health_and_income.csv")